### Validation

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cvxopt

In [5]:
class LinearRegression():
    def __init__(self):
        pass
    
    def fit(self, X, y):
        y = y.reshape(-1, 1)
        self.w = np.linalg.inv(X.T @ X) @ (X.T) @ y
        
    def predict(self, X):
        pred = X @ self.w
        return pred.squeeze()
    
def transform(X, funcs):
    """
    Inputs:
    - X: input nd-array
    - funcs: a list of functions applied on input's columns
    
    Return: a new input matrix with new features
    """
    
    for f in funcs:
        newcol = f(X).reshape(-1,1)
        X = np.hstack((X, newcol))
    return X

def error(y_true, y_pred):
    mul = y_true * y_pred
    return np.sum(mul < 0) / len(y_true)

In [6]:
inp = pd.read_table('datasets/in.dta', sep='  ', header=None, engine='python')
out = pd.read_table('datasets/out.dta', sep='  ', header=None, engine='python')

In [7]:
X, y = inp[[0,1]].to_numpy(), inp[2].to_numpy()
X_test, y_test = out[[0, 1]].to_numpy(), out[2].to_numpy()

X = np.hstack((np.ones((X.shape[0],1)), X))
X_test = np.hstack((np.ones((X_test.shape[0],1)), X_test))

nonlinear_trans = [lambda x: x[:,1]**2,
                  lambda x: x[:,2]**2,
                  lambda x: x[:,1]*x[:,2],
                  lambda x: np.abs(x[:,1]-x[:,2]),
                  lambda x: np.abs(x[:,1]+x[:,2])]

#### Question 1, 2

In [8]:
X_train, y_train = X[:25], y[:25]
X_val, y_val = X[25:], y[25:]


print('Running LR for different k')
for i in range(1, len(nonlinear_trans)+1):
    f = nonlinear_trans[:i]
    X_train_trans = transform(X_train, f)
    X_val_trans = transform(X_val, f)
    X_test_trans = transform(X_test, f)
    k = X_train.shape[1] + i - 1
    
    lr = LinearRegression()
    lr.fit(X_train_trans, y_train)
    pred_val = lr.predict(X_val_trans)
    pred_test = lr.predict(X_test_trans)
    Eval = error(y_val, pred_val)
    Eout = error(y_test, pred_test)
    print('k = {}, Eval = {}, Eout = {}'.format(k, Eval, Eout))

Running LR for different k
k = 3, Eval = 0.3, Eout = 0.42
k = 4, Eval = 0.5, Eout = 0.416
k = 5, Eval = 0.2, Eout = 0.188
k = 6, Eval = 0.0, Eout = 0.084
k = 7, Eval = 0.1, Eout = 0.072


#### Question 3, 4

In [9]:
X_val, y_val = X[:25], y[:25]
X_train, y_train = X[25:], y[25:]


print('Running LR for different k')
for i in range(1, len(nonlinear_trans)+1):
    f = nonlinear_trans[:i]
    X_train_trans = transform(X_train, f)
    X_val_trans = transform(X_val, f)
    X_test_trans = transform(X_test, f)
    k = X_train.shape[1] + i - 1
    
    lr = LinearRegression()
    lr.fit(X_train_trans, y_train)
    pred_val = lr.predict(X_val_trans)
    pred_test = lr.predict(X_test_trans)
    Eval = error(y_val, pred_val)
    Eout = error(y_test, pred_test)
    print('k = {}, Eval = {}, Eout = {}'.format(k, Eval, Eout))

Running LR for different k
k = 3, Eval = 0.28, Eout = 0.396
k = 4, Eval = 0.36, Eout = 0.388
k = 5, Eval = 0.2, Eout = 0.284
k = 6, Eval = 0.08, Eout = 0.192
k = 7, Eval = 0.12, Eout = 0.196


#### Question 6

In [10]:
e1_list = []
e2_list = []
e_list = []
for i in range(1000):
    e1 = np.random.uniform(0, 1)
    e2 = np.random.uniform(0, 1)
    e = np.min([e1, e2])
    e1_list.append(e1)
    e2_list.append(e2)
    e_list.append(e)
    
print('E[e1] = {}, E[e2] = {}, E[e] = {}'.format(np.mean(e1_list), np.mean(e2_list), np.mean(e_list)))

E[e1] = 0.49504150904424105, E[e2] = 0.49332181047097867, E[e] = 0.3295354387805394


#### Question 7

In [11]:
p = [np.sqrt(np.sqrt(3)+4), np.sqrt(np.sqrt(3)-1), np.sqrt(9+4*np.sqrt(6)), np.sqrt(9-np.sqrt(6))]
X = np.array([-1, 1]).reshape(-1,1)
y = np.array([0, 0])
for i in range(len(p)):
    X_copy = np.vstack((X, p[i]))
    y_copy = np.hstack((y, 1))
    e0 = []
    e1 = []
    for fold in range(3):
        X_train = X_copy[[idx for idx in range(len(X_copy)) if idx!=fold]]
        y_train = y_copy[[idx for idx in range(len(y_copy)) if idx!=fold]]
        X_val = X_copy[fold]
        y_val = y_copy[fold]
        X_train = np.hstack((np.ones((X_train.shape[0],1)), X_train))
        X_val = np.hstack((np.ones((X_val.shape[0])), X_val))
        
        h0 = np.mean(y_train)
        h1 = LinearRegression()
        h1.fit(X_train, y_train)
        pred_0 = h0
        pred_1 = h1.predict(X_val)
        e0.append((pred_0 - y_val)**2)
        e1.append((pred_1 - y_val)**2)
        
    print('p{}: Eout_0 = {}, Eout_1 = {}'.format(i, np.mean(e0), np.mean(e1)))

p0: Eout_0 = 0.5, Eout_1 = 1.135043367685941
p1: Eout_0 = 0.5, Eout_1 = 64.66494840795369
p2: Eout_0 = 0.5, Eout_1 = 0.5000000000000001
p3: Eout_0 = 0.5, Eout_1 = 0.9868839293305486


#### Question 8

In [12]:
def generate_line():
    """
    Generate a random line with corresponding coefficient and intercept
    """
    
    line_points = np.random.uniform(-1, 1, [2, 2])
    line_coef = (line_points[1,1] - line_points[0, 1])/(line_points[1,0] - line_points[0, 0])
    line_intercept = line_points[0,1] - line_coef*line_points[0,0]
    return line_coef, line_intercept

def generate_data(N, line_coef, line_intercept):
    """
    Generate random data given number of points, a line coefficient and intercept. Generated points will be in +1/-1 class regarding their relative position to the line.
    
    Inputs:
    - N: number of points to be generated
    - line_coef: line's coefficient
    - line_intercept: line's intercept
    
    Outputs: a tuple of 2 variables:
    - X: positions of generated points
    - y: classes of points (+1/-1)
    """
    
    X = np.random.uniform(-1, 1, [N, 2])
    y = X[:, 1] - X[:, 0] * line_coef - line_intercept >= 0
    y = np.where(y, 1, -1)
    if np.abs(np.sum(y)) == len(y):
        X, y = generate_data(N, line_coef, line_intercept)
    return X, y

In [13]:
class PLA():
    """
    PLA model
    """
    def __init__(self):
        pass
    
    def fit(self, X, y):
        self.w = np.zeros((X.shape[1],1))
        converged = False
        while not converged:
            converged = True
            for i in range(X.shape[0]):
                yi = y[i]
                xi = X[i].reshape(1,-1)
                if yi * (xi @ self.w) <= 0:
                    converged = False
                    self.w += (yi * xi).T
                    
    def predict(self, X):
        pred = np.sign(X @ self.w)
        return pred.squeeze()
    
class SVM():
    """
    SVM classifier
    """
    
    def __init__(self):
        self.w = None
        self.alpha = None
    
    def fit(self, X, y, form='primal'):
        self.form = form
        y = y.reshape(-1,1)
        cvxopt.solvers.options['show_progress'] = False
        if self.form == 'primal':
            """
            Find w directly
            """
            P = np.eye(X.shape[1])
            P[0,0] = 0 # we need to find w, b to minimize w, so the 1st value mapping to b is 0
            q = np.zeros(X.shape[1])
            G = -y * X
            h = -np.ones((len(y)))
            
            P = cvxopt.matrix(P)
            q = cvxopt.matrix(q)
            G = cvxopt.matrix(G)
            h = cvxopt.matrix(h)
            cvx = cvxopt.solvers.qp(P, q, G, h)
            self.w = np.array(cvx['x'])
            self.alpha = None
            
        if self.form == 'dual':
            """
            Find support vectors, then find w
            """
            X = X[:,1:]
            P = (y @ y.T) * (X @ X.T)
            q = -np.ones(len(y))
            G = -np.eye(len(y))
            h = np.zeros((len(y),1))
            A = (y.T) * np.ones(y.T.shape)
            b = np.zeros(1)
            
            P = cvxopt.matrix(P)
            q = cvxopt.matrix(q)
            G = cvxopt.matrix(G)
            h = cvxopt.matrix(h)
            A = cvxopt.matrix(A)
            b = cvxopt.matrix(b)
            cvx = cvxopt.solvers.qp(P, q, G, h, A, b)
            self.alpha = np.around(np.array(cvx['x']), decimals = 3)
            self.w = (y * X).T @ self.alpha
            ys = y[self.alpha > 0]
            Xs = X[self.alpha.squeeze() > 0]
            
            # KKT condition
            b = ys[0] - self.w.T @ Xs[0]
            self.w = np.vstack((b, self.w))
            
            
    def predict(self, X):
        return np.sign(X @ self.w).squeeze()

#### Question 8

In [14]:
cnt_svm = 0
n_runs = 1000
N_in = 10
N_out = 5000
for n in range(n_runs):
    coef, intercept = generate_line()
    X, y = generate_data(N_in, coef, intercept)
    X = np.hstack((np.ones((X.shape[0],1)), X))
    X_out, y_out = generate_data(N_out, coef, intercept)
    X_out = np.hstack((np.ones((X_out.shape[0],1)), X_out))
    
    pla = PLA()
    pla.fit(X, y)
    pla_pred = pla.predict(X_out)
    
    svm = SVM()
    svm.fit(X, y)
    svm_pred = svm.predict(X_out)
    
    e_pla = error(y_out, pla_pred)
    e_svm = error(y_out, svm_pred)
    if e_svm < e_pla:
        cnt_svm += 1
        
print('Percentage of times SVM outperforms PLA: {:.2f}%'.format(cnt_svm/n_runs*100))

Percentage of times SVM outperforms PLA: 63.30%


#### Question 9

In [15]:
cnt_svm = 0
n_runs = 1000
N_in = 100
N_out = 10000
for n in range(n_runs):
    coef, intercept = generate_line()
    X, y = generate_data(N_in, coef, intercept)
    X = np.hstack((np.ones((X.shape[0],1)), X))
    X_out, y_out = generate_data(N_out, coef, intercept)
    X_out = np.hstack((np.ones((X_out.shape[0],1)), X_out))
    
    pla = PLA()
    pla.fit(X, y)
    pla_pred = pla.predict(X_out)
    
    svm = SVM()
    svm.fit(X, y)
    svm_pred = svm.predict(X_out)
    
    e_pla = error(y_out, pla_pred)
    e_svm = error(y_out, svm_pred)
    if e_svm < e_pla:
        cnt_svm += 1
        
print('Percentage of times SVM outperforms PLA: {:.2f}%'.format(cnt_svm/n_runs*100))

Percentage of times SVM outperforms PLA: 62.90%


#### Question 10

In [16]:
cnt_svm = 0
n_runs = 1000
N_in = 100
sup_vecs = []
for n in range(n_runs):
    coef, intercept = generate_line()
    X, y = generate_data(N_in, coef, intercept)
    X = np.hstack((np.ones((X.shape[0],1)), X))
    
    svm = SVM()
    svm.fit(X, y, form='dual')
    sup_vecs.append(len(svm.alpha[svm.alpha>0]))
        
print('Average number of support vectors: {}'.format(np.mean(sup_vecs)))

Average number of support vectors: 3.002
